    Data Science Cancer Treatment - Aplicação de Deep Learning usando Python e Keras

In [ ]:
#Executei no cmd para monitorar
#!pip install tensorflow

In [ ]:
#Executei no cmd para monitorar
#!pip install keras

In [1]:
import scipy

In [2]:
from scipy.sparse import issparse

In [3]:
# Importa os pacotes
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
from keras.layers import Dense

In [5]:
from keras.optimizers import SGD

In [55]:
from keras.preprocessing.text import text_to_word_sequence

In [129]:
from keras.preprocessing.text import one_hot

In [217]:
from keras.preprocessing.text import hashing_trick

In [221]:
from keras.preprocessing.text import Tokenizer

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
import numpy as np

In [10]:
import pandas as pd

In [12]:
import matplotlib.pyplot as plt

In [192]:
#Carga dos datasets
ds_test_text = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/test_text_original", delimiter="\|\|", engine="python")

In [193]:
ds_test_text

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung cance...
2,2,Abstract Background Non-small cell lung cance...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...
...,...,...
133,133,The RNA maturation is an important and complex...
134,134,The splicing factor SF3B1 is the most commonly...
135,135,Mutations in genes encoding proteins involved ...
136,136,Precursor mRNA splicing is catalyzed by the sp...


In [194]:
ds_training_text = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/training_text_original", delimiter="\|\|", engine="python")

In [195]:
ds_training_text

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...
...,...,...
133,133,The RNA maturation is an important and complex...
134,134,The splicing factor SF3B1 is the most commonly...
135,135,Mutations in genes encoding proteins involved ...
136,136,Precursor mRNA splicing is catalyzed by the sp...


In [196]:
ds_test_variants = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/test_variants", delimiter=",", engine="python")

In [197]:
ds_test_variants

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A
...,...,...,...
5663,5663,SLC46A1,R113S
5664,5664,FOXC1,L130F
5665,5665,GSS,R267W
5666,5666,CTSK,G79E


In [198]:
ds_training_variants = pd.read_table("/FCD/MachineLearning/Cap23/Projetos-7-8/Data-Sci-DL-Python-Cancer-Treatment/data_files/training_variants", delimiter=",", engine="python")

In [199]:
ds_training_variants

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4
...,...,...,...,...
3316,3316,RUNX1,D171N,4
3317,3317,RUNX1,A122*,1
3318,3318,RUNX1,Fusions,1
3319,3319,RUNX1,R80C,4


PRÉ-PROCESSAMENTO

In [200]:
#Join dos datasets de textos e de variantes para treinamento
ds_training = pd.merge(ds_training_text, ds_training_variants, left_on="ID", right_on="ID")

In [201]:
ds_training

,ID,Text,Gene,Variation,Class
0,0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1
1,1,Abstract Background Non-small cell lung canc...,CBL,W802*,2
2,2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2
3,3,Recent evidence has demonstrated that acquired...,CBL,N454D,3
4,4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4
...,...,...,...,...,...
133,133,The RNA maturation is an important and complex...,SF3B1,K666N,9
134,134,The splicing factor SF3B1 is the most commonly...,SF3B1,K666R,9
135,135,Mutations in genes encoding proteins involved ...,SF3B1,K700E,9
136,136,Precursor mRNA splicing is catalyzed by the sp...,SF3B1,R957Q,9


In [202]:
#Join dos datasets de textos e de variantes para teste
ds_test = pd.merge(ds_test_text, ds_test_variants, left_on="ID", right_on="ID")

In [203]:
ds_test

,ID,Text,Gene,Variation
0,0,Cyclin-dependent kinases (CDKs) regulate a var...,ACSL4,R570S
1,1,Abstract Background Non-small cell lung cance...,NAGLU,P521L
2,2,Abstract Background Non-small cell lung cance...,PAH,L333F
3,3,Recent evidence has demonstrated that acquired...,ING1,A148D
4,4,Oncogenic mutations in the monomeric Casitas B...,TMEM216,G77A
...,...,...,...,...
133,133,The RNA maturation is an important and complex...,VCP,R159H
134,134,The splicing factor SF3B1 is the most commonly...,SAR1B,S179R
135,135,Mutations in genes encoding proteins involved ...,BIN1,R154Q
136,136,Precursor mRNA splicing is catalyzed by the sp...,FOXC1,S131L


ANÁLISE EXPLORATÓRIA

In [204]:
pd.isnull(ds_training)

,ID,Text,Gene,Variation,Class
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
133,False,False,False,False,False
134,False,False,False,False,False
135,False,False,False,False,False
136,False,False,False,False,False


In [205]:
pd.isnull(ds_test)

,ID,Text,Gene,Variation
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
133,False,False,False,False
134,False,False,False,False
135,False,False,False,False
136,False,False,False,False


In [206]:
pd.isna(ds_training)

,ID,Text,Gene,Variation,Class
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
133,False,False,False,False,False
134,False,False,False,False,False
135,False,False,False,False,False
136,False,False,False,False,False


In [207]:
pd.isna(ds_test)

,ID,Text,Gene,Variation
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
133,False,False,False,False
134,False,False,False,False
135,False,False,False,False
136,False,False,False,False


In [208]:
ds_training.dtypes

ID            int64
Text         object
Gene         object
Variation    object
Class         int64
dtype: object

In [209]:
ds_test.dtypes

ID            int64
Text         object
Gene         object
Variation    object
dtype: object

In [210]:
ds_training.size

690

In [211]:
ds_test.size

552

In [212]:
# Transformando os documentos em tokens
df_tokens = pd.DataFrame({'Words' : [],'Text' : []})

df_tokens['Words'] = ds_training.apply(lambda row: text_to_word_sequence(row.Text), axis = 1) 
df_tokens['Text'] = ds_training.apply(lambda row: row.Text, axis = 1) 

In [213]:
df_tokens

,Words,Text
0,"[cyclin, dependent, kinases, cdks, regulate, a...",Cyclin-dependent kinases (CDKs) regulate a var...
1,"[abstract, background, non, small, cell, lung,...",Abstract Background Non-small cell lung canc...
2,"[abstract, background, non, small, cell, lung,...",Abstract Background Non-small cell lung canc...
3,"[recent, evidence, has, demonstrated, that, ac...",Recent evidence has demonstrated that acquired...
4,"[oncogenic, mutations, in, the, monomeric, cas...",Oncogenic mutations in the monomeric Casitas B...
...,...,...
133,"[the, rna, maturation, is, an, important, and,...",The RNA maturation is an important and complex...
134,"[the, splicing, factor, sf3b1, is, the, most, ...",The splicing factor SF3B1 is the most commonly...
135,"[mutations, in, genes, encoding, proteins, inv...",Mutations in genes encoding proteins involved ...
136,"[precursor, mrna, splicing, is, catalyzed, by,...",Precursor mRNA splicing is catalyzed by the sp...


In [214]:
# Transformando as palavras dos documentos em inteiros
df_int = pd.DataFrame(columns = ['Int'])

for index, row in df_tokens.iterrows():
#estima o tamanho do vocabulário
    words = set(row['Words'])
    print(len(words)):


SyntaxError: invalid syntax (<ipython-input-214-87cb3d3fae40>, line 7)

In [215]:
# Transformando as palavras dos documentos em inteiros
df_int = pd.DataFrame(columns = ['Int'])

for index, row in df_tokens.iterrows():
#estima o tamanho do vocabulário
    words = set(row['Words'])
    vocab_size = len(words)
#codifica o documento com inteiros
    df_int = df_int.append({'Int': one_hot(row['Text'], vocab_size)}, ignore_index=True)

In [216]:
df_int

,Int
0,"[116, 847, 545, 90, 47, 696, 46, 310, 1362, 16..."
1,"[911, 972, 342, 1288, 971, 958, 498, 933, 1187..."
2,"[911, 972, 342, 1288, 971, 958, 498, 933, 1187..."
3,"[1144, 897, 989, 1219, 341, 614, 316, 943, 382..."
4,"[1231, 1046, 602, 20, 148, 386, 725, 724, 963,..."
...,...
133,"[1028, 411, 842, 52, 1294, 1321, 89, 1341, 143..."
134,"[902, 428, 420, 1501, 2083, 902, 895, 1290, 71..."
135,"[2508, 2644, 4755, 2903, 3615, 1290, 2644, 707..."
136,"[1141, 205, 239, 782, 845, 749, 1130, 592, 591..."


In [ ]:
#define the document
text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of the vocabulary
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
result = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
print(result)

In [218]:
# Transformando as palavras dos documentos em inteiros com hashing trick para garantir unicidade
df_intht = pd.DataFrame(columns = ['Int'])

for index, row in df_tokens.iterrows():
#estima o tamanho do vocabulário
    words = set(row['Words'])
    vocab_size = len(words)
#codifica o documento com inteiros
    df_intht = df_intht.append({'Int': hashing_trick(row['Text'], round(vocab_size*1.3), hash_function='md5')}, ignore_index=True)

In [219]:
df_intht

,Int
0,"[740, 560, 1879, 818, 1237, 113, 162, 990, 313..."
1,"[1276, 720, 1541, 1574, 699, 925, 972, 59, 100..."
2,"[1276, 720, 1541, 1574, 699, 925, 972, 59, 100..."
3,"[519, 152, 365, 666, 1542, 120, 1129, 1774, 40..."
4,"[1578, 1042, 992, 1611, 1881, 1876, 1307, 216,..."
...,...
133,"[1424, 1026, 1864, 652, 1195, 1579, 1177, 1439..."
134,"[855, 376, 2493, 1154, 2300, 855, 1641, 864, 9..."
135,"[954, 5908, 1805, 4420, 5237, 1106, 5908, 3432..."
136,"[628, 1455, 74, 1059, 1308, 966, 830, 1152, 68..."


AS FUNÇÕES DE PRÉ-PROCESSAMENTO DO KERAS ANTERIORES FORAM UTILIZADAS COMO EXERCÍCIO UMA VEZ QUE SÃO MELHORES PARA UM ÚNICO REGISTRO.
ABAIXO API DO KERAS PARA MULTIPLOS DOCUMENTOS, O QUE MELHOR SE APLICA A ESTE CASO

In [ ]:
# define 5 documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

In [223]:
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(ds_training.Text)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(ds_training.Text, mode='count')
print(encoded_docs)

OrderedDict([('cyclin', 1929), ('dependent', 533), ('kinases', 369), ('cdks', 79), ('regulate', 139), ('a', 18267), ('variety', 100), ('of', 49702), ('fundamental', 29), ('cellular', 388), ('processes', 134), ('cdk10', 129), ('stands', 2), ('out', 453), ('as', 7238), ('one', 1771), ('the', 59846), ('last', 119), ('orphan', 3), ('for', 10577), ('which', 3081), ('no', 1537), ('activating', 271), ('has', 1795), ('been', 2322), ('identified', 1451), ('and', 39417), ('kinase', 1239), ('activity', 2554), ('revealed', 525), ('previous', 435), ('work', 141), ('shown', 1699), ('that', 11633), ('silencing', 96), ('increases', 88), ('ets2', 83), ('v', 373), ('ets', 103), ('erythroblastosis', 2), ('virus', 100), ('e26', 2), ('oncogene', 108), ('homolog', 68), ('2', 5424), ('driven', 87), ('activation', 1005), ('mapk', 212), ('pathway', 1005), ('confers', 18), ('tamoxifen', 24), ('resistance', 109), ('to', 20524), ('breast', 736), ('cancer', 4466), ('cells', 7544), ('precise', 50), ('mechanisms', 3

[[0.000e+00 2.540e+02 2.110e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 2.810e+02 1.840e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 2.810e+02 1.840e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [0.000e+00 1.891e+03 1.761e+03 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 2.390e+02 2.750e+02 ... 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 1.620e+02 2.160e+02 ... 1.000e+00 1.000e+00 1.000e+00]]


In [225]:
# Garante a reproducividade do código
seed = 7
np.random.seed(seed)

In [ ]:
#seguir script C:\FCD\MachineLearning\Cap14\2-01-Cap12-Keras